In [6]:

def rename_documents(dataPath):

    """ dataPath : "20_newsgroups/" """
    new_path = os.getcwd()+'/'+dataPath
    every_path = list(os.walk(new_path))
    
    total = 1
    for i in range(0,len(every_path)):
        dirPath, dirName, fileNames = every_path[i]
        if(len(dirName) == 0):
            for j in fileNames:
                single_doc_loc = dirPath + '/' + j
                os.rename(single_doc_loc, dirPath + '/' + str(total))
                total+=1
    
    return     

In [ ]:
def testDataTokens(test_doc):
    test_docs = []
    
    for doc in test_doc:
        test_docs.append(tokenizeDocument(doc))
        
    return test_docs

In [7]:
def collect_documents(dataPath):

    docs = {}
    labels = []
    
    new_path = os.getcwd()+'/'+dataPath
    every_path = list(os.walk(new_path))
    
    f = 0
    for i in range(0,len(every_path)):
        dirPath, dirName, fileNames = every_path[i]
        if(len(dirName) == 0):
            for j in fileNames:
                single_doc = []
                single_doc_loc = dirPath + '/' + j
                with open(single_doc_loc, 'rb') as f:
                    single_doc.append(str(f.read()))
                docs[j] = single_doc
                labels.append(dirPath.split('/').pop())
    return docs, labels

In [ ]:
#### Inverted Index ####

def makeIndex(token_docid, C):
    inverted_index = {}   # {token:[df,{'class i': 'token occurence count in class i'}]}
    class_index = {}   # {class i: total no. of tokens }
    
    for c in range(0, C):
        class_index[c] = 0
        
    for i in range(0, token_docid.shape[0]):
        term = token_docid.loc[i, 'token']
        doclabel = token_docid.loc[i, 'class']
        
        if(term not in inverted_index.keys()):
            postings_list = [0,{}]  # df, key = docid, value = tf
            postings_list[0] = 1
            postings_list[1][doclabel] = 1 #tf 
            inverted_index[term] = postings_list
            class_index[doclabel]+=1
            
        else:
            plist = inverted_index[term] 
            if doclabel not in plist[1].keys():
                plist[1][doclabel] = 1 #tf 
                plist[0]+=1
                class_index[doclabel]+=1
            else:
                plist[1][doclabel]+=1
                class_index[doclabel]+=1

    return inverted_index, class_index


In [ ]:
#### Inverted Index ####

def makeIndexQ2(token_docid):
    inverted_index = {}   # {token:[df,cf, {'doc_id': 'token occurence count in document doc_id'}]}
    
    for i in range(0, token_docid.shape[0]):
        term = token_docid.loc[i, 'token']
        docid = token_docid.loc[i, 'doc_id']
        
        if(term not in inverted_index.keys()):
            postings_list = [0,0,{}]  # df, key = docid, value = tf
            postings_list[0] = 1
            postings_list[1]+=1
            postings_list[2][docid] = 1 #tf 
            inverted_index[term] = postings_list
        else:
            plist = inverted_index[term] 
            if docid not in plist[2].keys():
                plist[2][docid] = 1 #tf 
                plist[0]+=1
                plist[1]+=1
            else:
                plist[2][docid]+=1
                plist[1]+=1

    return inverted_index

In [ ]:
def tokenizeDocument(docs, qt = False):
    """
        Args:
            docs (dict) : all documents
        return:
            final tokens (list) : terms for index

        0. Convert to lowercase
        1. Stop words removed
        2. Tokenize 
        3. Stemming
        4. Lemmatization
        5. Only words that starts with alphabet or digit. Front 0's removed.
    """

    stop_ws = set(stopwords.words('english'))

    ts = docs.split('\\n')
    docs = ' '.join(ts)
    ts = docs.split('\t')

    docs = ' '.join(ts)
  
    # Tokenization
    tokens = WordPunctTokenizer().tokenize(docs)
    
    ntokens = []
    tokenizer = RegexpTokenizer(r'[^_]')
    for t in tokens:
        ntokens.append(''.join(tokenizer.tokenize(t)))

    # lowercase
    tokens_lowercase = [ w.lower() for w in ntokens]
    
    #Remove Stop words
    tokens_stop  = [ w for w in tokens_lowercase if(w not in stop_ws)] 
    
#     tokens_stem = [ PorterStemmer().stem(w) for w in tokens_stop]
    
    # Lemmatization
    updated_tokens = [ WordNetLemmatizer().lemmatize(w) for w in tokens_stop]
     
    final_tokens = []
    updated_tokens = list(filter(None, updated_tokens))
    
    for updated_token in updated_tokens:
        if(updated_token.isalpha() and len(updated_token) > 1 and  d.check(updated_token)):
            final_tokens.append(updated_token)
#         else:
#             if(updated_token.isnumeric()):
#                 final_tokens.append(str(int(updated_token)))
                
    if(not qt):
        final_tokens = final_tokens[1:]  # remove b
    else:
        return final_tokens  

    return final_tokens
      

In [ ]:
def splitDataset(data, a):
    """
    Randomly split dataset
    """
    nr = int((a/100)*(data.shape[0]/C))
    train_data = data.groupby('class').head(nr)
    test_data = pd.concat([data, train_data]).drop_duplicates(keep=False)

    return train_data, test_data

In [ ]:
def convertLabels(train_data, test_data):
    doc_type_ct = train_data['class'].value_counts()
    print(doc_type_ct)
    docs_type = sorted(doc_type_ct.index.tolist())
    
    print(docs_type)
    
    priors_probab = [ doc_type_ct[i]/doc_type_ct.sum() for i in docs_type]
    print(priors_probab)
    # convert labels to numbers
    train_data['class'].replace(docs_type, [0,2,3,4,1], inplace = True)
    test_data['class'].replace(docs_type, [0,2,3,4,1], inplace = True)
    
    return train_data, test_data, priors_probab

In [ ]:
def trainNB(inverted_index):
    """
        Calculate p(xi | y) for each feature for each class label in the training data
    """
    likelihood = np.zeros((C, len(inverted_index.keys())))
    total_tokens = len(inverted_index.keys())
    
    t = 0
    for token, plist in inverted_index.items():
        ll = []
        for c in range(0,C):
            if(c in plist[1].keys()):
                ll.append((plist[1][c] + 1)/(class_index[c] + total_tokens))
            else:
                ll.append(1/(class_index[c] + total_tokens))
        
        likelihood[:, t] = ll
        t+=1

    likelihoods = pd.DataFrame(likelihood, columns = list(inverted_index.keys()))
    return likelihoods


In [ ]:
def findAccuracy(predicted_class, test_data):
    cm = np.zeros((C,C))
    
    for i in range(0, len(predicted_class)):
        cm[predicted_class[i], test_data.loc[i, 'class']]+=1

    return cm

In [1]:
def featureSelect(index, total_docs):
    term_util = []
    
    out = 0
    for token, plist in index.items():
        clear_output(wait = True)
        df = plist[0]
        for doc,tf in plist[2].items():
            tf_idf = (1 + math.log(plist[1],10)) * math.log(total_docs/df,10)
            term_util.append([token, tf_idf])
        print('Current progress: ', np.round((out/len(inverted_index.keys()))*100, 2))
        out+=1
        
    return term_util
        

In [ ]:
def drawConfusionMatrix(cm, title = False):
    predict_ll = ['comp.graphics', 'rec.sport.hockey', 'sci.med', 'sci.space', 'talk.politics.misc']
#     actual_ll = [ 'Actual '+str(i) for i in range(0, C)]
    
    hm = pd.DataFrame(cm, index = predict_ll, columns = predict_ll)
    sns.heatmap(hm, annot = True, fmt='g', cmap = "YlGnBu")
    plt.title('Confusion Matrix for '+title)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.show()
    return
        

In [ ]:
def runQ2(feat, reduced_vocab_terms):
   
    new_inv = {}
    
    for token in reduced_vocab_terms:
        new_inv[token] = inverted_index[token]
    
    new_likelihood = trainNB(new_inv)
    new_predicts = classify(test_docs, new_inv, new_likelihood)
    new_conf_mat = findAccuracy(new_predicts, test_data)
    
    acc = (np.trace(new_conf_mat)/np.sum(new_conf_mat))*100
    print('Accuracy for ',str(split_percentage) +':'+ str(100 - split_percentage), ' with top '+str(feat)+'% features: ', str(acc)+'%.')
#     drawConfusionMatrix(new_conf_mat, 'top ' + str(feat) +'% features')
    return acc
    